## NMF analysis on FES cohort

### get top10k std cpg

In [ ]:
## python code
import pandas as pd

df = pd.read_csv("00.data/comb.data.patient.csv", sep=",", low_memory=False)
cpg_std_df = df.std(axis=1)
sorted_df = cpg_std_df.sort_values(ascending=False)
top10k = sorted_df.head(10000).index.to_list()
top10k_df = df.loc[top10k]
top10k_df.to_csv("02.result/NMF/comb.patient.top10k.tsv", sep="\t")

### extract features relevant to subtype identification in FES cohort

In [ ]:
## R code 
library(NMF)

patient.df <- read.table("02.result/NMF/comb.patient.top10k.tsv", sep="\t", header=TRUE, row.names=1)
res <- nmf(patient.df, 2, .opt='vp20')
score <- featureScore(res)
write.table(score, "treatment.Comb.patient.top10k.feature.score.csv", sep=",")
s <- extractFeatures(res)
write.table(as.data.frame(s[2][[1]]), "02.result/NMF/onset.Comb.patient.top10k.feature.cluster_1.csv", sep=",", row.names=FALSE)
write.table(as.data.frame(s[1][[1]]), "02.result/NMF/onset.Comb.patient.top10k.feature.cluster_2.csv", sep=",", row.names=FALSE)

In [ ]:
## python code
import pandas as pd
## extract CpG site name
onset_feature_dmp_1 = pd.read_csv("02.result/NMF/onset/onset.Comb.patient.top10k.feature.cluster_1.csv")
onset_feature_dmp_2 = pd.read_csv("02.result/NMF/onset/onset.Comb.patient.top10k.feature.cluster_2.csv")

onset_feature_cpg_1 = onset_score_df.reset_index().loc[onset_feature_dmp_1['s[2][[1]]']-1, "index"].values.tolist()
onset_feature_cpg_2 = onset_score_df.reset_index().loc[onset_feature_dmp_2['s[1][[1]]']-1, "index"].values.tolist()

pd.DataFrame(onset_feature_cpg_1+onset_feature_cpg_2).to_csv("02.result/NMF/onset/onset.Comb.patient.top10k.feature.all.cpg.csv", index=False, header=False)

## extract methylation matrix in FES and LTS cohorts according to FES relevant features

In [ ]:
## R code
treat.sample.df <- read.table("treatment.sample.sheet.csv", header=TRUE, sep=",")
treat.mycomb <- readRDS("treat.comb.clean.rds")
treat.patient.df <- sample.df[sample.df$Sample_Group_x==2,]
onset.cpg <- read.table("02.result/NMF/onset/onset.Comb.patient.top10k.feature.all.cpg.csv")

## select CpG sites valid in both cohorts
valid.cpg <- intersect(onset.cpg$V1), rownames(treat.mycomb))
treat.patient.cpg <- treat.mycomb[valid.cpg, as.vector(treat.patient.df$Sample_Name)]
write.table(treat.patient.cpg, "02.result/NMF/onset.NMF.feature.treatment.patient.cpg.csv", sep=",", quote=FALSE)

onset.comb <- readRDS("comb.data.rds")
onset.df <- read.table("sample.info.csv", header=TRUE, sep=",")
onset.patient.df <- onset.df[onset.df$Sample_Group=="patient",]
onset.patient.cpg <- onset.comb[valid.cpg, as.vector(onset.patient.df$Sample_Name)]
write.table(onset.patient.cpg, "02.result/NMF/onset.NMF.feature.onset.patient.cpg.csv", sep=",", quote=FALSE)

## Train RandomForest model on FES cohort and test on LTS cohort

In [ ]:
## python code
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

def get_train(cpg_df, meta_df):
    X = cpg_df.T.loc[meta_df["Sample_Name"]]
    y = meta_df["nmf_cluster"]

    classifier = RandomForestClassifier(max_depth=20, random_state=0)
    classifier.fit(X, y)
    return classifier

## read in methylation matrix and sample information
onset_cpg_df = pd.read_csv("02.result/NMF/onset.NMF.feature.onset.patient.cpg.csv")
treat_cpg_df = pd.read_csv("02.result/NMF/onset.NMF.feature.treatment.patient.cpg.csv")

treat_meta_df = pd.read_csv("03.result/treatment.patient.NMF.csv")
treat_meta_df["nmf_cluster"].replace({"NMF_P1": 1, "NMF_P2":0, "Health":1}, inplace=True)
treat_patient_df = treat_meta_df[treat_meta_df["Sample_Group_x"]==2]

onset_meta_df = pd.read_csv("02.result/NMF/onset.patient.NMF.csv")
onset_meta_df["nmf_cluster"].replace({"Meth_P1": 1, "Meth_P2":0, "Health":1}, inplace=True)
onset_patient_df = onset_meta_df[onset_meta_df["Sample_Group"]=="patient"]

## build, train and test the RF model
onset_classifier = get_train(onset_cpg_df, onset_patient_df)

X_test = treat_cpg_df.T.loc[treat_patient_df["Sample_Name"]]
y_test = treat_patient_df["nmf_cluster"].tolist()

result_ls = []
result_ls.append(treat_patient_df["Sample_Name"].tolist())
result_ls.append(list(onset_classifier.predict(X_test)))
result_ls.append(y_test)
result_df = pd.DataFrame(result_ls).T
result_df.rename(columns={0: "Sample_Name", 1: "onset_prediction", 2: "treatment_cluster"}, inplace=True)
result_df.replace({1: "P1", 0: "P2"}, inplace=True)

result_df.to_csv("02.result/NMF/onset.NMF.feature.treatment.cross_classification.RF.prediction.csv", index=False)